In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [53]:
# 기존 데이터 읽기
df = pd.read_excel('dashboard.xlsx')  # 기존 엑셀 파일명 변경

# 새로운 데이터 읽기
new_data = pd.read_excel('dashboard_0922.xlsx')  # 새로운 데이터 엑셀 파일명 변경

In [54]:
# '제품군' 열에서 앞의 4글자 제거
df['E:P'] = df['E:P'].astype(str).str[4:]

# '구분2', '구분3' 열의 결측치를 '제안'으로 채움
df['구분2'] = df['구분2'].fillna('제안')
df['구분3'] = df['구분3'].fillna('제안')

# '구분2', '구분3' 열의 결측치를 '제안'으로 채움
new_data['구분2'] = new_data['구분2'].fillna('제안')
new_data['구분3'] = new_data['구분3'].fillna('제안')

In [55]:
# 기존 데이터 전처리
X = df[['구분2']]
y = df[['담당팀', '보고 구분']]


In [56]:
# 라벨 인코딩
X = df[['구분2']].copy()  # 슬라이스의 명확한 복사본 생성
le = LabelEncoder()
X.loc[:,'구분2'] = le.fit_transform(X['구분2'])

In [57]:
# 모델 학습
model = RandomForestClassifier()
model.fit(X, y)

RandomForestClassifier()

In [58]:
# 새로운 데이터에 대해 처리
for index, row in new_data.iterrows():
    if row['구분2'] == '제안':
        # '구분2'가 '제안'일 때
        new_data.at[index, '담당팀'] = row['E:P']  # '담당팀'에 'E:P'의 데이터 입력
        new_data.at[index, '보고 구분'] = '제안'  # '보고 구분'에 '제안' 입력
    elif pd.isnull(row['담당팀']) or pd.isnull(row['보고 구분']):
        # '담당팀' 또는 '보고 구분'이 비어있을 때만 모델을 사용하여 예측
        input_value = row['구분2']
        
        if input_value in le.classes_:
            input_data = pd.DataFrame({'구분2': [input_value]})
            input_data['구분2'] = le.transform(input_data['구분2'])
            
            # 모델 예측
            prediction = model.predict(input_data)[0]
            
            # 예측 결과를 기존 데이터 구조에 맞게 업데이트
            new_data.at[index, '담당팀'] = prediction[0]  # 예측된 담당팀 값 입력
            new_data.at[index, '보고 구분'] = prediction[1]  # 예측된 보고 구분 값 입력


In [59]:
# 새로운 엑셀 파일에 예측 결과 업데이트 (기존 파일 덮어쓰기)
new_data.to_excel('새로운데이터.xlsx', index=False)